# 1. Create a database DB to store objects A, B and C each in a separate table

In [1]:
import sqlite3
import json
conn = sqlite3.connect("my_database.db")
cursor = conn.cursor()
table_names = ["A", "B", "C"]
for table_name in table_names:
    with open(f"{table_name}.json", "r") as f:
        data = json.load(f)
    columns = ",".join(data[0].keys())
    cursor.execute(f"CREATE TABLE {table_name} ({columns})")
    for row in data:
        values = ",".join([f"'{value}'" for value in row.values()])
        cursor.execute(f"INSERT INTO {table_name} VALUES ({values})")
conn.commit()
conn.close()


In [2]:
conn = sqlite3.connect("my_database.db")
cursor = conn.cursor()
table_names = ["A", "B", "C"]
for table_name in table_names:
    cursor.execute(f"SELECT * FROM {table_name}")
    print(f"\nТаблица {table_name}:")
    for row in cursor.fetchall():
        print(row)
conn.close()



Таблица A:
('1', 'red')
('2', 'green')
('-2', 'blue')

Таблица B:
('prod', '2')
('sum', '1')
('pow', '-2')

Таблица C:
('[2, 0, 0]',)
('[0, 3, 0]',)
('[0, 0, -0.25]',)


# Создание пула функций

In [3]:
def sum_func(a, b):
    return a + b

def sub_func(a, b):
    return a - b

def pow_func(a, b):
    return a ** b

def prod_func(a, b):
    return a * b

def div_func(a, b):
    return a / b
func_map = {
    "sum": sum_func,
    "sub": sub_func,
    "pow": pow_func,
    "prod": prod_func,
    "div": div_func,
}

# 2. Implement a function that takes A and B and return C, where B.function applies to A.value and B.value and the result is stored in C.value with an index that depends on A.color according to the map:

In [4]:
def process_data(a, b, func_map):
    if not hasattr(a, "value") or not hasattr(a, "color"):
        raise ValueError("Объект A должен иметь поля 'value' и 'color'")
    if not hasattr(b, "value") or not hasattr(b, "function"):
        raise ValueError("Объект B должен иметь поля 'value' и 'function'")
    color_index_map = {"red": 0, "green": 1, "blue": 2}
    try:
        index = color_index_map[a.color]
    except KeyError:
        raise ValueError(f"Неизвестный цвет: {a.color}")
    if isinstance(b.function, str):
        try:
            b.function = func_map[b.function]
        except KeyError:
            raise ValueError(f"Неизвестное сокращение функции: {b.function}")
    result = b.function(a.value, b.value)
    c = [0] * 3
    c[index] = result
    return c


In [5]:
a = type("A", (), {"value": -2, "color": "blue"})
b = type("B", (), {"value": -2, "function": "pow"})
c = process_data(a, b, func_map)
print(c)  

[0, 0, 0.25]


# 3. Implement a function similar to function-2 but with INPUT from DB and OUTPUT to DB

In [6]:
conn = sqlite3.connect("my_database.db")
cursor = conn.cursor()
cursor.execute("SELECT value, color FROM A")
a_data = cursor.fetchall()
cursor.execute("SELECT function, value FROM B")
b_data = cursor.fetchall()
objects_a = []
for row in a_data:
    obj = type("ObjectA", (object,), {"value": float(row[0]), "color": row[1]})
    objects_a.append(obj)
objects_b = []
for row in b_data:
    obj = type("ObjectB", (object,), {"function": row[0], "value": float(row[1])})
    objects_b.append(obj)

for obj_a, obj_b in zip(objects_a, objects_b):
    if obj_b.function == "prod":
        obj_b.value = float(obj_b.value)  
    result = process_data(obj_a, obj_b, func_map)
    
    print(f" {result}")
conn.commit()
conn.close()

 [2.0, 0, 0]
 [0, 3.0, 0]
 [0, 0, 0.25]


In [7]:
conn = sqlite3.connect("my_database.db")
cursor = conn.cursor()
cursor.execute("SELECT value, color FROM A")
a_data = cursor.fetchall()
cursor.execute("SELECT function, value FROM B")
b_data = cursor.fetchall()
objects_a = []

for row in a_data:
    obj = type("ObjectA", (object,), {"value": float(row[0]), "color": row[1]})
    objects_a.append(obj)

objects_b = []
for row in b_data:
    obj = type("ObjectB", (object,), {"function": row[0], "value": float(row[1])})
    objects_b.append(obj)

results = []
for obj_a, obj_b in zip(objects_a, objects_b):
    if obj_b.function == "prod":
        obj_b.value = float(obj_b.value)
    result = process_data(obj_a, obj_b, func_map)
    results.append(result)

conn_new = sqlite3.connect("my_database.db")
cursor_new = conn_new.cursor()

cursor_new.execute("""
CREATE TABLE IF NOT EXISTS results (
    value_json TEXT
)
""")

for result in results:
    value_json = json.dumps(result)
    cursor_new.execute("INSERT INTO results (value_json) VALUES (?)", (value_json,))


conn_new.commit()
conn_new.close()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

for table_name in tables:
    print(f"Таблица: {table_name[0]}")
    cursor.execute(f"SELECT * FROM {table_name[0]}")
    data = cursor.fetchall()
    for row in data:
        print(row)

conn.commit()
conn.close()

Таблица: A
('1', 'red')
('2', 'green')
('-2', 'blue')
Таблица: B
('prod', '2')
('sum', '1')
('pow', '-2')
Таблица: C
('[2, 0, 0]',)
('[0, 3, 0]',)
('[0, 0, -0.25]',)
Таблица: results
('[2.0, 0, 0]',)
('[0, 3.0, 0]',)
('[0, 0, 0.25]',)
